In [10]:
import BioSimSpace as BSS
import sys
import networkx as nx
import pickle
import itertools as it

import warnings
warnings.filterwarnings(action='once')


try:
    import pipeline
except:
    print("adding code to the pythonpath...")
    code = '/home/anna/Documents/code/python'
    if code not in sys.path:
        sys.path.insert(1, code)
    import pipeline

from pipeline import *
from pipeline.analysis import *

In [7]:
for prot, net in it.product(["tyk2","p38","mcl1"],["rbfenn","lomap"]):

results_files = ["/home/anna/Documents/code/test/final_summary_SOMD_MBAR_alchemlyb_benchmark.csv"]
engine = "SOMD"
output_folder = "/home/anna/Documents/code/test"
net_file = f"/home/anna/Documents/benchmark/{prot}_benchmark/execution_model/network_{net}.dat"
weight_file = f"/home/anna/Documents/benchmark/{prot}_benchmark/execution_model/network_{net}_scores.dat"

perturbations, ligands, mod_results_files = get_info_network(results_files, net_file, extra_options={"engine":engine})

graph_x = graph(ligands, perturbations)
graph_x.add_weight(weight_file)
# graph_x.draw_graph()
avg_weighted = graph_x.get_average_weighted_simple_paths()
print(f"{prot} {net} is {avg_weighted}")

with open(f"{output_folder}/bound_pmf_{prot}_{net}.pickle", 'wb') as handle:
    pickle.dump(graph_x, handle)

using input file to get the weights
36.08531894778112


In [8]:
# cycles = graph.cycle_closures()
# print(cycles)

nx.get_edge_attributes(graph_x.graph, "weight")

print(len(perturbations))

48


In [ ]:
with open(f"{self._pickle_dir}/bound_pmf_{pickle_ext}.pickle", "rb") as file:
                    self._bound_pmf_dict = pickle.load(file)